In [1]:
import pandas as pd
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import CountVectorizer

C:\ProgramData\Anaconda3\lib\site-packages\scipy\__init__.py:155: UserWarning: A NumPy version >=1.18.5 and <1.25.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
df = pd.read_csv("diffusion_db_unaltered.csv")

In [6]:
nsfw = df[df['prompt_nsfw'] > 0.5]
nsfw.head()

,prompt,image_nsfw,prompt_nsfw
186,"portrait of a cloaked female devil, evil, omin...",0.184256,0.526609
187,"portrait of a cloaked female devil, evil, omin...",0.093387,0.537081
188,"portrait of a cloaked female devil, evil, omin...",0.176597,0.535589
191,"portrait of a cloaked female devil, evil, omin...",0.864233,0.536021
766,"kpop idol woman dressed in pink, eating a suck...",0.186505,0.873680


In [13]:
harmful = []
for prompt in nsfw['prompt']:
    for word in prompt.split():
        if word not in harmful:
            harmful.append(word)
    break

In [14]:
harmful

['portrait',
 'of',
 'a',
 'cloaked',
 'female',
 'devil,',
 'evil,',
 'ominous,',
 'luscious,',
 'pointy',
 'teeth,',
 'stunning,',
 'detailed,',
 'by',
 'artgerm,',
 'greg',
 'rutkowski,',
 'luis',
 'royo,',
 'pixar,',
 'myazaki,',
 'gothic,',
 'final',
 'fantasy,',
 'medieval']

In [3]:
df1 = df[:1000]
df1.loc[df['prompt_nsfw'] >= 0.5, "prompt_nsfw"] = "Harmful"
df1.loc[df['prompt_nsfw']  < 0.5, "prompt_nsfw"] = "Harmless"

C:\Users\sspindt\AppData\Local\Temp\ipykernel_35228\4009812897.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1.loc[df['prompt_nsfw'] >= 0.5, "prompt_nsfw"] = "Harmful"


In [4]:
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(df1['prompt'])

In [5]:
y = df1['prompt_nsfw']

In [6]:
neigh = KNeighborsClassifier(n_neighbors=11)
clf = MultinomialNB()

In [7]:
# KNN Fit
neigh.fit(X, y)

# Bayes Fit
clf.fit(X, y)

MultinomialNB()

In [9]:
# Predict with KNN
y_pred = neigh.predict(X)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\neighbors\_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  mode, _ = stats.mode(_y[neigh_ind, k], axis=1)


In [8]:
# Predict with Bayes
y_pred_bayes = clf.predict(X)

In [10]:
y_pred_bayes

array(['Harmless', 'Harmless', 'Harmless', 'Harmless', 'Harmless',
       'Harmless', 'Harmless', 'Harmless', 'Harmless', 'Harmless',
       'Harmless', 'Harmless', 'Harmless', 'Harmless', 'Harmless',
       'Harmless', 'Harmless', 'Harmless', 'Harmless', 'Harmless',
       'Harmless', 'Harmless', 'Harmless', 'Harmless', 'Harmless',
       'Harmless', 'Harmless', 'Harmless', 'Harmless', 'Harmless',
       'Harmless', 'Harmless', 'Harmless', 'Harmless', 'Harmless',
       'Harmless', 'Harmless', 'Harmless', 'Harmless', 'Harmless',
       'Harmless', 'Harmless', 'Harmless', 'Harmless', 'Harmless',
       'Harmless', 'Harmless', 'Harmless', 'Harmless', 'Harmful',
       'Harmless', 'Harmless', 'Harmless', 'Harmful', 'Harmful',
       'Harmless', 'Harmless', 'Harmless', 'Harmless', 'Harmless',
       'Harmless', 'Harmless', 'Harmful', 'Harmless', 'Harmless',
       'Harmless', 'Harmless', 'Harmless', 'Harmless', 'Harmless',
       'Harmless', 'Harmless', 'Harmless', 'Harmless', 'Harmless',

In [11]:
from sklearn.metrics import accuracy_score
accuracy_score(y, y_pred_bayes)

0.983

In [12]:
from sklearn.metrics import precision_score
precision_score(y, y_pred_bayes, average='macro')

0.4974696356275304

In [13]:
from sklearn.metrics import recall_score
recall_score(y, y_pred_bayes, average='macro')

0.49396984924623116

In [14]:
from sklearn.metrics import f1_score
f1_score(y, y_pred_bayes, average='macro')

0.4957135653050933